In [1]:
import csv
import lzstring
from collections import namedtuple, Counter
import json
from memoize import memoize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# [num unique urls, num unique urls typed, total visits, total typed, first visit time, last visit time]
domaininfo = namedtuple('domaininfo', ['num_unique_urls', 'num_unique_urls_typed', 'total_visits', 'total_typed', 'first_visit_time', 'last_visit_time'])

decompressFromEncodedURIComponent = lzstring.LZString().decompressFromEncodedURIComponent

#filepath = 'difficultyselectionexp_may31_11am.csv'
filepath = 'difficultyselectionexp_june6_3pm.csv'
reader = csv.DictReader(open(filepath))

def extract_domain_visit_info(domain_visit_info_compressed):
  domain_visit_info = json.loads(decompressFromEncodedURIComponent(domain_visit_info_compressed))
  output = {}
  for k,v in domain_visit_info.items():
    linedata = domaininfo(*v)
    output[k] = linedata
  return output

alldata = []

for alldata_item in reader:
  if alldata_item['selected_difficulty'] not in ['nothing', 'easy', 'medium', 'hard']:
    continue
  if alldata_item['domain_visit_info_compressed'] == None or len(alldata_item['domain_visit_info_compressed']) == 0:
    continue
  alldata_item['domain_visit_info'] = extract_domain_visit_info(alldata_item['domain_visit_info_compressed'])
  alldata.append(alldata_item)


In [3]:
training_data = alldata[:round(len(alldata)*0.8)]
test_data = alldata[round(len(alldata)*0.8):]
print(len(training_data))
print(len(test_data))

146
37


In [4]:
def extract_labels_alldata(data):
  return np.array([line['selected_difficulty'] for line in data])

@memoize
def get_most_common_label():
  label_to_count = Counter()
  for line in training_data:
    label = line['selected_difficulty']
    label_to_count[label] += 1
  sorted_by_count = sorted(label_to_count.items(), key=lambda x: x[1], reverse=True)
  return sorted_by_count[0][0]

@memoize
def get_most_visited_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += info.total_visits
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

@memoize
def get_most_common_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

def get_num_visits_for_domain(domain_visit_info, domain):
  info = domain_visit_info.get(domain, None)
  if info != None:
    return info.total_visits
  return 0

def extract_features_for_user(domain_visit_info):
  domains = get_most_common_domains()
  visits_for_domains = np.array([get_num_visits_for_domain(domain_visit_info, x) for x in domains])
  visits_for_domains = np.divide(visits_for_domains, np.sum(visits_for_domains))
  return visits_for_domains

def extract_features_alldata(data):
  output = []
  for line in data:
    domain_visit_info = line['domain_visit_info']
    features = extract_features_for_user(domain_visit_info)
    output.append(features)
  return np.array(output)


In [5]:
def get_percent_correct(predicted_labels, actual_labels):
  if len(predicted_labels) != len(actual_labels):
    raise 'need predicted and actual labels to have same lengths'
  total = len(actual_labels)
  correct = 0
  for p,a in zip(predicted_labels, actual_labels):
    if p == a:
      correct += 1
  return correct / total

def test_baseline_classifier():
  most_common_label = get_most_common_label()
  predictions = [most_common_label for line in test_data]
  actual = extract_labels_alldata(test_data)
  percent_correct = get_percent_correct(predictions, actual)
  print('baseline classifier accuracy:', percent_correct)

def test_classifier(clf, str=None):
  actual = extract_labels_alldata(test_data)
  features_test = extract_features_alldata(test_data)
  predictions = clf.predict(features_test)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier testing accuracy:', percent_correct)

def training_error_classifier(clf, str=None):
  actual = extract_labels_alldata(training_data)
  features_train = extract_features_alldata(training_data)
  predictions = clf.predict(features_train)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier training accuracy:', round(percent_correct, 2))

def to_int_categorical(dt):
  # {'easy', 'hard', 'medium', 'nothing'}
  cat_dt = []
  for item in dt:
    if item == 'nothing':
      cat_dt.append(0)
    elif item == 'easy':
      cat_dt.append(1)
    elif item == 'medium':
      cat_dt.append(2)
    else:
      cat_dt.append(3)
  return np.array(cat_dt)
    
test_baseline_classifier()

baseline classifier accuracy: 0.5945945945945946


In [6]:
labels_train = extract_labels_alldata(training_data)
features_train = extract_features_alldata(training_data)

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(features_train, labels_train)
test_classifier(clf, 'RF')

clf = KNeighborsClassifier(n_neighbors=3, p=1)
clf.fit(features_train, labels_train)
test_classifier(clf, 'KNN')

RF classifier testing accuracy: 0.3783783783783784
KNN classifier testing accuracy: 0.5135135135135135


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical

labels_test = to_categorical(to_int_categorical(extract_labels_alldata(test_data)), num_classes=4)
features_test = extract_features_alldata(test_data)

labels_train = to_categorical(to_int_categorical(extract_labels_alldata(training_data)), num_classes=4)
features_train = extract_features_alldata(training_data)

model = Sequential()

model.add(Dense(100, activation='relu', input_dim=100))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(features_train, labels_train,
          epochs=100,
          batch_size=32)
score = model.evaluate(features_test, labels_test, batch_size=32)

predictions = model.predict(features_test, batch_size=32)

print(predictions)
print(labels_test)
print(score)

Using TensorFlow backend.


Epoch 1/100
146/146 [==============================] - 0s 810us/step - loss: 1.3933 - acc: 0.2534
Epoch 2/100
146/146 [==============================] - 0s 69us/step - loss: 1.3641 - acc: 0.4178
Epoch 3/100
146/146 [==============================] - 0s 69us/step - loss: 1.3470 - acc: 0.4247
Epoch 4/100
146/146 [==============================] - 0s 81us/step - loss: 1.3228 - acc: 0.4041
Epoch 5/100
146/146 [==============================] - 0s 87us/step - loss: 1.3016 - acc: 0.4315
Epoch 6/100
146/146 [==============================] - 0s 92us/step - loss: 1.2879 - acc: 0.4315
Epoch 7/100
146/146 [==============================] - 0s 67us/step - loss: 1.2822 - acc: 0.4452
Epoch 8/100
146/146 [==============================] - 0s 104us/step - loss: 1.2922 - acc: 0.4315
Epoch 9/100
146/146 [==============================] - 0s 91us/step - loss: 1.2689 - acc: 0.4384
Epoch 10/100
146/146 [==============================] - 0s 76us/step - loss: 1.2697 - acc: 0.4315
Epoch 11/100
146/146 [=====

146/146 [==============================] - 0s 70us/step - loss: 1.2507 - acc: 0.4658
Epoch 83/100
146/146 [==============================] - 0s 63us/step - loss: 1.2370 - acc: 0.4247
Epoch 84/100
146/146 [==============================] - 0s 59us/step - loss: 1.2359 - acc: 0.4726
Epoch 85/100
146/146 [==============================] - 0s 88us/step - loss: 1.2398 - acc: 0.4384
Epoch 86/100
146/146 [==============================] - 0s 61us/step - loss: 1.2405 - acc: 0.4521
Epoch 87/100
146/146 [==============================] - 0s 75us/step - loss: 1.2341 - acc: 0.4384
Epoch 88/100
146/146 [==============================] - 0s 82us/step - loss: 1.2243 - acc: 0.4589
Epoch 89/100
146/146 [==============================] - 0s 68us/step - loss: 1.2272 - acc: 0.4452
Epoch 90/100
146/146 [==============================] - 0s 70us/step - loss: 1.2382 - acc: 0.4589
Epoch 91/100
146/146 [==============================] - 0s 73us/step - loss: 1.2449 - acc: 0.4589
Epoch 92/100
146/146 [===========